In [ ]:
import h5py as h5
import numpy as np
import pandas as pd
from tqdm import tqdm
from copy import deepcopy


from multiresticodm.config import Config
from multiresticodm.inputs import Inputs
from multiresticodm.outputs import Outputs
from multiresticodm.utils.misc_utils import *
from multiresticodm.utils.math_utils import *
from multiresticodm.utils.probability_utils import *
from multiresticodm.contingency_table import instantiate_ct
from multiresticodm.markov_basis import instantiate_markov_basis

In [ ]:
%matplotlib inline

# AUTO RELOAD EXTERNAL MODULES
%load_ext autoreload
%autoreload 2

In [ ]:
def validate_tables(out):
    if out.inputs is None:
        out.inputs = Inputs(
            config = out.config,
            synthetic_data = False,
            logger = out.logger
        )
    else:
        try:
            out.inputs.cast_from_xarray()
        except:
            pass
    ct = instantiate_ct(
        config = out.config,
        **out.inputs.data_vars(),
        level = 'EMPTY'
    )
    samples = out.get_sample('table')
    print(dict(samples.sizes))
    print('axes constraints',ct.constraints['constrained_axes'])
    print('cell constraints',len(ct.constraints['cells']))
    tables_admissible = all([ct.table_admissible(torch.tensor(tab.values.squeeze())) for _,tab in samples.groupby('id')])
    print('Tables admissible',tables_admissible)
    if not tables_admissible:
        print('Tables margins admissible',any([ct.table_margins_admissible(torch.tensor(tab.values.squeeze())) for _,tab in samples.groupby('id')]))
        print('Tables cells admissible',all([ct.table_cells_admissible(torch.tensor(tab.values.squeeze())) for _,tab in samples.groupby('id')]))

In [ ]:
# Get important paths
experiment_id = 'NonJointTableSIM_NN_SweepedNoise_26_01_2024_13_26_18'
# 'NonJointTableSIM_NN_SweepedNoise_26_01_2024_13_26_18'
# 'NonJointTableSIM_NN_SweepedNoise_30_01_2024_23_25_12'
# 'NonJointTableSIM_NN_SweepedNoise_01_02_2024_00_45_29'
# 'JointTableSIM_NN_SweepedNoise_23_01_2024_21_33_25'
# 'JointTableSIM_NN_SweepedNoise_30_01_2024_21_29_31'
experiment_dir = f'../data/outputs/cambridge_work_commuter_lsoas_to_msoas/exp1/{experiment_id}/'
relative_experiment_dir = os.path.relpath(experiment_dir,os.getcwd())

In [ ]:
# Output processing settings
settings = {
    "logging_mode": "INFO",
    "coordinate_slice": [
        "da.loss_name == str(['dest_attraction_ts_likelihood_loss'])",
        "~da.title.isin([str('_unconstrained'), str('_total_constrained')])"
    ],
    "metadata_keys":[],
    "burnin_thinning_trimming": [{'iter': {"burnin":10000, "thinning":90, "trimming":1000}}],
    "n_workers": 1,
    "group_by":["seed"],
    "filename_ending":"test",
    "sample":["table"],
    "force_reload":False
}

In [ ]:
sweep_id = 'loss_name_[dest_attraction_ts_likelihood_loss]/seed_23/'
sigmas = ['low','high','learned']
titles = ['_doubly_constrained','_doubly_10%_cell_constrained','_doubly_20%_cell_constrained']
progress = tqdm(
    total = len(sigmas)*len(titles),
    desc = 'Loading sweep data'
)
for sig in sigmas:
    for titl in titles:
        # Create current sweep id
        current_sweep_id = os.path.join('samples',sweep_id,f"sigma_{sig}/title_{titl}/")
        
        print(f"sigma_{sig}/title_{titl}/")
        # Initialise outputs
        current_sweep_outputs = Outputs(
            config = os.path.join(relative_experiment_dir,current_sweep_id),
            settings = settings,
            inputs = None,
            slice = True,
            level = 'INFO'
        )
        # Silence outputs
        current_sweep_outputs.logger.setLevels(console_level='EMPTY')
        # Load all data
        current_sweep_outputs.load()
        # Get first collection id
        current_sweep_outputs0 = current_sweep_outputs.get(0)
        # Validate tables
        # validate_tables(current_sweep_outputs0)
        print('SRMSE',srmse(current_sweep_outputs0.data.table.mean('id'),current_sweep_outputs.inputs.data.ground_truth_table).values.squeeze())
        print('\n')
        # break
    # break

In [ ]:
# Get important paths
experiment_id = 'NonJointTableSIM_NN_SweepedNoise_01_02_2024_00_45_29'
# 'NonJointTableSIM_NN_SweepedNoise_26_01_2024_13_26_18'
# 'NonJointTableSIM_NN_SweepedNoise_30_01_2024_23_25_12'
# 'NonJointTableSIM_NN_SweepedNoise_01_02_2024_00_45_29'
# 'JointTableSIM_NN_SweepedNoise_23_01_2024_21_33_25'
# 'JointTableSIM_NN_SweepedNoise_30_01_2024_21_29_31'
experiment_dir = f'../data/outputs/cambridge_work_commuter_lsoas_to_msoas/exp1/{experiment_id}/'
relative_experiment_dir = os.path.relpath(experiment_dir,os.getcwd())

In [ ]:
# Initialise outputs
current_sweep_outputs = Outputs(
    config = relative_experiment_dir,
    settings = settings,
    inputs = None,
    slice = True,
    level = 'INFO'
)
# Silence outputs
current_sweep_outputs.logger.setLevels(console_level='EMPTY')
# Load all data
current_sweep_outputs.load()
# Get first collection id
# current_sweep_outputs0 = current_sweep_outputs.get(0)
# Validate tables
# validate_tables(current_sweep_outputs0)

# Temporary scripts

In [ ]:
experiment_id = 'JointTableSIM_NN_SweepedNoise_23_01_2024_21_33_25'
experiment_dir = f'../data/outputs/cambridge_work_commuter_lsoas_to_msoas/exp1/{experiment_id}/'
relative_experiment_dir = os.path.relpath(experiment_dir,os.getcwd())

In [ ]:
for d in os.walk(relative_experiment_dir):
    if 'doubly' in d[0]:
        for f in ['data.h5','metadata.json','outputs.log']:
            if os.path.exists(os.path.join(d[0],f)):
                os.remove(os.path.join(d[0],f))
        if os.path.exists(d[0]):
            os.rmdir(d[0])

In [ ]:
for sweep_config in tqdm(current_sweep_outputs.config.sweep_configurations,total=len(current_sweep_outputs.config.sweep_configurations)):
    new_config,sweep = current_sweep_outputs.config.prepare_experiment_config(sweep_config)
    sweep_id = current_sweep_outputs.config.get_sweep_id(sweep = sweep)
    filepath = os.path.join(relative_experiment_dir,'samples',sweep_id,'metadata.json')
    metadata = read_json(filepath = filepath)
    metadata['neural_network']['loss'] = new_config['neural_network']['loss']
    write_json(data = metadata, filepath = filepath)
